# Case Study 3: DSO Intervention Strategies

This notebook examines how different Distribution System Operator (DSO) intervention strategies and policies affect agent behavior, market participation, and grid stability in decentralized local energy markets.

## 📋 Table of Contents

1. [Research Questions & Hypothesis](#research-questions--hypothesis)
2. [Setup & Imports](#setup--imports)
3. [Scenario Configuration](#scenario-configuration)
4. [DSO Intervention Strategies](#dso-intervention-strategies)
5. [Agent Creation](#agent-creation)
6. [Training & Evaluation](#training--evaluation)
7. [Grid Stability Analysis](#grid-stability-analysis)
8. [Results Analysis](#results-analysis)
9. [Research Implications](#research-implications)

---

## 🔬 Research Questions & Hypothesis

### Research Questions Addressed:
- How do different DSO intervention thresholds affect market participation?
- What is the impact of DSO pricing policies on agent strategies?
- How do grid constraints enforced by DSO influence coordination?
- What balance between market freedom and grid stability is optimal?
- How do agents adapt to varying levels of DSO strictness?

### Hypothesis:
More strict DSO intervention will improve grid stability but may reduce market efficiency and agent participation, while lenient policies may lead to grid constraints but higher economic efficiency.

### DSO Intervention Strategies Tested:
1. **Permissive:** Minimal intervention, high market freedom
2. **Moderate:** Balanced intervention with standard thresholds
3. **Strict:** High intervention with strict grid constraints
4. **Dynamic:** Adaptive intervention based on real-time conditions


## 🛠️ Setup & Imports

Let's import all necessary libraries and set up the environment for our DSO intervention analysis.


In [ ]:
# Standard library imports
import sys
import os
import warnings
from dataclasses import dataclass
from typing import Any, Dict, List
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Imports successful!")
print(f"📁 Project root: {project_root}")
print(f"🐍 Python version: {sys.version}")
print(f"📊 NumPy version: {np.__version__}")
print(f"📈 Pandas version: {pd.__version__}")


In [ ]:
# Import project-specific modules
try:
    from src.agent.battery import Battery
    from src.agent.der import DERAgent
    from src.grid.network import GridNetwork, GridTopology
    from src.market.dso import DSOAgent
    from src.market.matching import MarketConfig
    from src.market.mechanism import ClearingMechanism
    from src.profile.der import DERProfileHandler
    from src.profile.dso import DSOProfileHandler
    from src.environment.train import RLTrainer, TrainingMode, RLAlgorithm
    
    print("✅ Project modules imported successfully!")
    
    # Display available clearing mechanisms
    print("\n📋 Available Clearing Mechanisms:")
    for mechanism in ClearingMechanism:
        print(f"  - {mechanism.name}: {mechanism.value}")
        
except ImportError as e:
    print(f"❌ Error importing project modules: {e}")
    print("Please ensure you're running this notebook from the correct directory")
    print("and that all dependencies are installed.")


## ⚙️ Scenario Configuration

Let's define the base configuration parameters for our DSO intervention analysis. These parameters will be kept constant across all intervention strategies to ensure fair comparison.


In [ ]:
@dataclass
class Case3Scenarios:
    """Case 3: DSO Intervention Strategies scenarios configuration."""
    
    # Base simulation parameters
    NUM_AGENTS = 8
    MAX_STEPS = 24  # 24-hour simulation
    GRID_CAPACITY = 1500.0  # kW
    
    # Market parameters
    MIN_PRICE = 40.0  # $/MWh
    MAX_PRICE = 250.0  # $/MWh
    MIN_QUANTITY = 0.1  # kWh
    MAX_QUANTITY = 200.0  # kWh

# Display configuration
print("📊 Case 3 Configuration:")
print(f"  Number of Agents: {Case3Scenarios.NUM_AGENTS}")
print(f"  Simulation Length: {Case3Scenarios.MAX_STEPS} hours")
print(f"  Grid Capacity: {Case3Scenarios.GRID_CAPACITY} kW")
print(f"  Price Range: ${Case3Scenarios.MIN_PRICE} - ${Case3Scenarios.MAX_PRICE} /MWh")
print(f"  Quantity Range: {Case3Scenarios.MIN_QUANTITY} - {Case3Scenarios.MAX_QUANTITY} kWh")
print(f"  DSO Strategies: 4 (Permissive, Moderate, Strict, Dynamic)")


## 🏛️ DSO Intervention Strategies

Now let's define the four different DSO intervention strategies to understand how regulatory approaches affect market dynamics and grid stability.


In [ ]:
def create_permissive_dso() -> DSOAgent:
    """Create permissive DSO with minimal intervention."""
    return DSOAgent(
        intervention_threshold=0.95,  # High threshold - intervene only at 95% capacity
        penalty_multiplier=1.2,        # Low penalty multiplier
        price_adjustment_factor=0.1,   # Minimal price adjustments
        grid_stability_weight=0.3,     # Low weight on grid stability
        market_efficiency_weight=0.7,  # High weight on market efficiency
        intervention_frequency=0.1,    # Low intervention frequency
        adaptive_threshold=False       # Fixed threshold
    )

def create_moderate_dso() -> DSOAgent:
    """Create moderate DSO with balanced intervention."""
    return DSOAgent(
        intervention_threshold=0.8,    # Moderate threshold - intervene at 80% capacity
        penalty_multiplier=1.5,        # Moderate penalty multiplier
        price_adjustment_factor=0.2,   # Moderate price adjustments
        grid_stability_weight=0.5,     # Balanced weights
        market_efficiency_weight=0.5, # Balanced weights
        intervention_frequency=0.3,    # Moderate intervention frequency
        adaptive_threshold=False       # Fixed threshold
    )

def create_strict_dso() -> DSOAgent:
    """Create strict DSO with high intervention."""
    return DSOAgent(
        intervention_threshold=0.7,    # Low threshold - intervene at 70% capacity
        penalty_multiplier=2.0,        # High penalty multiplier
        price_adjustment_factor=0.3,   # High price adjustments
        grid_stability_weight=0.8,     # High weight on grid stability
        market_efficiency_weight=0.2,  # Low weight on market efficiency
        intervention_frequency=0.5,    # High intervention frequency
        adaptive_threshold=False       # Fixed threshold
    )

def create_dynamic_dso() -> DSOAgent:
    """Create dynamic DSO with adaptive intervention."""
    return DSOAgent(
        intervention_threshold=0.8,    # Base threshold
        penalty_multiplier=1.5,        # Base penalty multiplier
        price_adjustment_factor=0.2,   # Base price adjustments
        grid_stability_weight=0.6,     # Slightly higher weight on stability
        market_efficiency_weight=0.4,  # Slightly lower weight on efficiency
        intervention_frequency=0.4,    # Higher intervention frequency
        adaptive_threshold=True        # Adaptive threshold based on conditions
    )

# Display DSO strategies
print("🏛️ DSO Intervention Strategies:")
print("=" * 80)

dso_strategies = {
    "Permissive": create_permissive_dso(),
    "Moderate": create_moderate_dso(),
    "Strict": create_strict_dso(),
    "Dynamic": create_dynamic_dso()
}

for strategy_name, dso in dso_strategies.items():
    print(f"\n{strategy_name} DSO:")
    print(f"  Intervention Threshold: {dso.intervention_threshold:.1%}")
    print(f"  Penalty Multiplier: {dso.penalty_multiplier:.1f}x")
    print(f"  Price Adjustment Factor: {dso.price_adjustment_factor:.1%}")
    print(f"  Grid Stability Weight: {dso.grid_stability_weight:.1%}")
    print(f"  Market Efficiency Weight: {dso.market_efficiency_weight:.1%}")
    print(f"  Intervention Frequency: {dso.intervention_frequency:.1%}")
    print(f"  Adaptive Threshold: {dso.adaptive_threshold}")

print("\n" + "=" * 80)


## 👥 Agent Creation

Now let's create diverse agent configurations that represent realistic market participants. We'll create 8 different agent types to ensure realistic market dynamics and test how different DSO intervention strategies affect various agent profiles.


In [ ]:
def create_standard_agents() -> List[DERAgent]:
    """Create standard agent configuration for DSO intervention analysis."""
    agents = []
    profile_handler = DERProfileHandler()
    
    print("🏗️ Creating agents for DSO intervention analysis...")
    
    # Mix of agent types to create realistic market stress
    agent_configs = [
        # High-generation agents (potential grid stress)
        {"id": "high_gen_001", "capacity": 90.0, "battery_cap": 45.0, "demand_mult": 0.8},
        {"id": "high_gen_002", "capacity": 85.0, "battery_cap": 40.0, "demand_mult": 0.7},
        
        # High-demand agents (potential grid stress)
        {"id": "high_dem_001", "capacity": 40.0, "battery_cap": 20.0, "demand_mult": 2.0},
        {"id": "high_dem_002", "capacity": 35.0, "battery_cap": 15.0, "demand_mult": 1.8},
        
        # Balanced prosumers
        {"id": "balanced_001", "capacity": 60.0, "battery_cap": 30.0, "demand_mult": 1.1},
        {"id": "balanced_002", "capacity": 55.0, "battery_cap": 25.0, "demand_mult": 1.2},
        
        # Flexible agents with larger batteries (good for grid services)
        {"id": "flexible_001", "capacity": 70.0, "battery_cap": 60.0, "demand_mult": 1.0},
        {"id": "flexible_002", "capacity": 65.0, "battery_cap": 55.0, "demand_mult": 1.1}
    ]
    
    for i, config in enumerate(agent_configs, 1):
        print(f"  Creating agent {i}/8: {config['id']}")
        
        generation, demand = profile_handler.get_energy_profiles(
            Case3Scenarios.MAX_STEPS,
            config["capacity"]
        )
        
        # Apply demand multiplier based on agent type
        demand = [d * config["demand_mult"] for d in demand]
        
        agent = DERAgent(
            id=config["id"],
            capacity=config["capacity"],
            battery=Battery(
                nominal_capacity=config["battery_cap"],
                min_soc=0.1,
                max_soc=0.9,
                charge_efficiency=0.94,
                discharge_efficiency=0.94
            ),
            generation_profile=generation,
            demand_profile=demand
        )
        agents.append(agent)
    
    print(f"✅ Created {len(agents)} agents successfully!")
    return agents


In [ ]:
# Create the standard agents
agents = create_standard_agents()

# Display agent summary
print("\n📊 Agent Summary:")
print("=" * 80)
for agent in agents:
    battery_info = f"Battery: {agent.battery.nominal_capacity:.1f} kWh" if agent.battery else "No Battery"
    print(f"ID: {agent.id:<15} | Capacity: {agent.capacity:>6.1f} kW | {battery_info}")
    
print("=" * 80)
total_capacity = sum(agent.capacity for agent in agents)
total_battery = sum(agent.battery.nominal_capacity for agent in agents if agent.battery)
print(f"Total Generation Capacity: {total_capacity:.1f} kW")
print(f"Total Battery Capacity: {total_battery:.1f} kWh")
print(f"Number of Agents: {len(agents)}")


## 🔄 Scenario Generation

Now let's create scenarios for each DSO intervention strategy to understand how different regulatory approaches affect market dynamics and grid stability.


In [ ]:
def create_base_grid_network() -> GridNetwork:
    """Create base grid network configuration using IEEE34 topology."""
    return GridNetwork(
        topology=GridTopology.IEEE34,
        num_nodes=Case3Scenarios.NUM_AGENTS,
        capacity=Case3Scenarios.GRID_CAPACITY,
        seed=42
    )

def create_base_market_config() -> MarketConfig:
    """Create base market configuration for DSO intervention analysis."""
    return MarketConfig(
        min_price=Case3Scenarios.MIN_PRICE,
        max_price=Case3Scenarios.MAX_PRICE,
        min_quantity=Case3Scenarios.MIN_QUANTITY,
        max_quantity=Case3Scenarios.MAX_QUANTITY,
        price_mechanism=ClearingMechanism.BID_ASK_SPREAD,  # Market-driven pricing
        enable_partner_preference=True,  # Enable strategic partner selection
        blockchain_difficulty=2,
        visualize_blockchain=False
    )

def get_all_scenarios() -> Dict[str, Dict[str, Any]]:
    """Generate all Case 3 scenarios for different DSO intervention strategies."""
    
    scenarios = {}
    agents = create_standard_agents()
    grid_network = create_base_grid_network()
    market_config = create_base_market_config()
    der_profile_handler = DERProfileHandler()
    
    print("🔄 Creating scenarios for all DSO intervention strategies...")
    
    # Create scenarios for each DSO strategy
    dso_strategy_configs = [
        ("permissive_dso", create_permissive_dso()),
        ("moderate_dso", create_moderate_dso()),
        ("strict_dso", create_strict_dso()),
        ("dynamic_dso", create_dynamic_dso())
    ]
    
    for i, (strategy_name, dso_agent) in enumerate(dso_strategy_configs, 1):
        print(f"  Creating scenario {i}/4: {strategy_name}")
        
        # Create DSO profile handler with specific DSO agent
        dso_profile_handler = DSOProfileHandler(
            min_price=Case3Scenarios.MIN_PRICE,
            max_price=Case3Scenarios.MAX_PRICE,
            dso_agent=dso_agent
        )
        
        scenario_config = {
            "max_steps": Case3Scenarios.MAX_STEPS,
            "agents": agents.copy(),  # Use copy to avoid shared state
            "market_config": market_config,
            "grid_network": grid_network,
            "der_profile_handler": der_profile_handler,
            "dso_profile_handler": dso_profile_handler,
            "enable_reset_dso_profiles": True,
            "enable_asynchronous_order": True,
            "max_error": 0.15,  # Moderate error tolerance for DSO intervention
            "num_anchor": 5,  # Moderate anchors for DSO analysis
            "seed": 42
        }
        
        scenarios[strategy_name] = scenario_config
    
    print(f"✅ Created {len(scenarios)} scenarios successfully!")
    return scenarios


In [ ]:
# Generate all scenarios
scenarios = get_all_scenarios()

# Display scenario summary
print("\n📊 Scenario Summary:")
print("=" * 80)
for scenario_name, config in scenarios.items():
    strategy_name = scenario_name.replace("_", " ").title()
    dso_agent = config['dso_profile_handler'].dso_agent
    print(f"Scenario: {scenario_name}")
    print(f"  Strategy: {strategy_name}")
    print(f"  Agents: {len(config['agents'])}")
    print(f"  Max Steps: {config['max_steps']}")
    print(f"  Price Range: ${config['market_config'].min_price} - ${config['market_config'].max_price} /MWh")
    print(f"  DSO Intervention Threshold: {dso_agent.intervention_threshold:.1%}")
    print(f"  DSO Penalty Multiplier: {dso_agent.penalty_multiplier:.1f}x")
    print()

print(f"Total scenarios created: {len(scenarios)}")
print("=" * 80)


## 🎯 Training & Evaluation

Now let's train each DSO intervention scenario to understand how different regulatory approaches affect agent learning, coordination, and grid stability.


### Agent Behavior Options

We provide two options for agent behavior:

1. **Zero Intelligence Agents (Default)** - Agents use uniform random distribution for bidding decisions, making it easier to visualize DSO intervention effects
2. **MARL Training** - Agents learn optimal strategies through reinforcement learning

The zero intelligence option serves as a baseline and makes it easier to observe the pure effects of different DSO intervention strategies without the complexity of learning dynamics.


In [ ]:
# Configuration: Choose agent behavior type
USE_ZERO_INTELLIGENCE = True  # Set to False for MARL training

print("🤖 Agent Behavior Configuration:")
print("=" * 50)
if USE_ZERO_INTELLIGENCE:
    print("✅ Using Zero Intelligence Agents (Default)")
    print("  • Uniform random distribution for bidding")
    print("  • Easier to visualize DSO intervention effects")
    print("  • No learning dynamics complexity")
    print("  • Faster execution for demonstration")
else:
    print("🧠 Using MARL Training")
    print("  • Agents learn optimal strategies")
    print("  • Reinforcement learning approach")
    print("  • More realistic agent behavior")
    print("  • Longer training time required")

print(f"\nCurrent setting: {'Zero Intelligence' if USE_ZERO_INTELLIGENCE else 'MARL Training'}")
print("=" * 50)


In [ ]:
# Training configuration
TRAINING_EPISODES = 200  # Reduced for demonstration
EVALUATION_EPISODES = 50
ALGORITHM = RLAlgorithm.PPO
TRAINING_MODE = TrainingMode.CTDE

# Store training results
training_results = {}

print(f"🎯 Training Configuration:")
print(f"  Algorithm: {ALGORITHM.name}")
print(f"  Training Mode: {TRAINING_MODE.name}")
print(f"  Training Episodes: {TRAINING_EPISODES}")
print(f"  Evaluation Episodes: {EVALUATION_EPISODES}")
print(f"  Scenarios to Train: {len(scenarios)}")
print(f"  Agent Behavior: {'Zero Intelligence' if USE_ZERO_INTELLIGENCE else 'MARL Training'}")
print()


In [ ]:
# Modified training section with zero intelligence option
if USE_ZERO_INTELLIGENCE:
    print("🚀 Running Zero Intelligence Agent Simulations...")
    print("=" * 80)
    
    # Store training results
    training_results = {}
    
    for i, (scenario_name, config) in enumerate(scenarios.items(), 1):
        print(f"\n📈 Running Scenario {i}/{len(scenarios)}: {scenario_name}")
        print("-" * 60)
        
        try:
            # For zero intelligence, we'll use proper environment stepping with random actions
            print(f"  🔄 Running zero intelligence simulation...")
            
            # Create trainer to get access to environment
            trainer = RLTrainer(
                env_config=config,
                algorithm=ALGORITHM,  # Algorithm doesn't matter for zero intelligence
                training=TRAINING_MODE,
                iters=1  # Minimal iterations since we're not training
            )
            
            # Reset environment
            trainer.env.reset()
            
            # Run simulation with random actions
            total_reward = 0.0
            episode_rewards = []
            
            for episode in range(10):  # Run 10 episodes for zero intelligence
                episode_reward = 0.0
                
                for step in range(config['max_steps']):
                    # Generate random valid actions for all agents
                    actions = {}
                    for agent_id in trainer.env.agents:
                        # Use action_spaces instead of action_space for DTDE mode
                        if hasattr(trainer.env, 'action_spaces') and trainer.env.action_spaces is not None:
                            action_space = trainer.env.action_spaces[agent_id]
                        else:
                            action_space = trainer.env.action_space[agent_id]
                        actions[agent_id] = action_space.sample()
                    
                    # Step the environment
                    obs, rewards, terminated, truncated, info = trainer.env.step(actions)
                    
                    # Accumulate rewards
                    step_reward = sum(rewards.values()) if isinstance(rewards, dict) else rewards
                    episode_reward += step_reward
                    
                    if terminated or truncated:
                        break
                
                episode_rewards.append(episode_reward)
                total_reward += episode_reward
                
                # Reset for next episode
                trainer.env.reset()
            
            # Calculate average performance
            avg_reward = total_reward / len(episode_rewards)
            final_reward = episode_rewards[-1] if episode_rewards else 0.0
            
            # Store results
            training_results[scenario_name] = {
                "trainer": trainer,
                "config": config,
                "status": "completed",
                "zero_intelligence": True,
                "final_reward": final_reward,
                "avg_reward": avg_reward,
                "episode_rewards": episode_rewards
            }
            
            print(f"  ✅ Zero intelligence simulation completed!")
            print(f"  📊 Final Reward: {final_reward:.3f}")
            print(f"  📊 Average Reward: {avg_reward:.3f}")
            print(f"  📊 Episodes Run: {len(episode_rewards)}")
            
        except Exception as e:
            print(f"  ❌ Simulation failed: {e}")
            training_results[scenario_name] = {
                "trainer": None,
                "config": config,
                "status": "failed",
                "error": str(e)
            }

else:
    print("🚀 Starting MARL training for all scenarios...")
    print("=" * 80)
    
    for i, (scenario_name, config) in enumerate(scenarios.items(), 1):
        print(f"\n📈 Training Scenario {i}/{len(scenarios)}: {scenario_name}")
        print("-" * 60)
        
        try:
            # Create trainer
            trainer = RLTrainer(
                env_config=config,
                algorithm=ALGORITHM,
                training=TRAINING_MODE,
                iters=TRAINING_EPISODES
            )
            
            # Train the scenario
            print(f"  🔄 Training with {ALGORITHM.name} algorithm...")
            trainer.train()
            
            # Store results
            training_results[scenario_name] = {
                "trainer": trainer,
                "config": config,
                "status": "completed",
                "zero_intelligence": False
            }
            
            print(f"  ✅ Training completed successfully!")
            
        except Exception as e:
            print(f"  ❌ Training failed: {e}")
            training_results[scenario_name] = {
                "trainer": None,
                "config": config,
                "status": "failed",
                "error": str(e),
                "zero_intelligence": False
            }

print("\n" + "=" * 80)
print("🎉 Training/Simulation completed for all scenarios!")
print(f"Successful: {sum(1 for r in training_results.values() if r['status'] == 'completed')}")
print(f"Failed: {sum(1 for r in training_results.values() if r['status'] == 'failed')}")


In [ ]:
# Training configuration
TRAINING_EPISODES = 200  # Reduced for demonstration
EVALUATION_EPISODES = 50
ALGORITHM = RLAlgorithm.PPO
TRAINING_MODE = TrainingMode.CTDE

print(f"🎯 Training Configuration:")
print(f"  Algorithm: {ALGORITHM.name}")
print(f"  Training Mode: {TRAINING_MODE.name}")
print(f"  Training Episodes: {TRAINING_EPISODES}")
print(f"  Evaluation Episodes: {EVALUATION_EPISODES}")
print(f"  Scenarios to Train: {len(scenarios)}")
print()

# Store training results
training_results = {}

print("🚀 Starting training for all DSO intervention strategies...")
print("=" * 80)

for i, (scenario_name, config) in enumerate(scenarios.items(), 1):
    print(f"\n📈 Training Scenario {i}/{len(scenarios)}: {scenario_name}")
    print("-" * 60)
    
    try:
        # Create trainer
        trainer = RLTrainer(
            env_config=config,
            algorithm=ALGORITHM,
            training=TRAINING_MODE,
            iters=TRAINING_EPISODES
        )
        
        # Train the scenario
        print(f"  🔄 Training with {ALGORITHM.name} algorithm...")
        trainer.train()
        
        # Store results
        training_results[scenario_name] = {
            "trainer": trainer,
            "config": config,
            "status": "completed"
        }
        
        print(f"  ✅ Training completed successfully!")
        
    except Exception as e:
        print(f"  ❌ Training failed: {e}")
        training_results[scenario_name] = {
            "trainer": None,
            "config": config,
            "status": "failed",
            "error": str(e)
        }

print("\n" + "=" * 80)
print("🎉 Training completed for all scenarios!")
print(f"Successful: {sum(1 for r in training_results.values() if r['status'] == 'completed')}")
print(f"Failed: {sum(1 for r in training_results.values() if r['status'] == 'failed')}")


## 📊 Grid Stability Analysis

Let's analyze how different DSO intervention strategies affect grid stability, market participation, and agent behavior.


In [ ]:
# Calculate grid stability and DSO intervention metrics
print("📊 Grid Stability Analysis")
print("=" * 80)

grid_stability_data = []

for scenario_name, config in scenarios.items():
    dso_agent = config['dso_profile_handler'].dso_agent
    agents = config['agents']
    
    # Calculate grid stress indicators
    total_capacity = sum(agent.capacity for agent in agents)
    total_battery = sum(agent.battery.nominal_capacity for agent in agents if agent.battery)
    
    # Calculate potential grid stress
    max_generation = max(agent.capacity for agent in agents)
    max_demand = max(max(agent.demand_profile) for agent in agents)
    
    # DSO intervention characteristics
    intervention_strictness = 1.0 - dso_agent.intervention_threshold  # Higher = stricter
    penalty_severity = dso_agent.penalty_multiplier
    stability_weight = dso_agent.grid_stability_weight
    efficiency_weight = dso_agent.market_efficiency_weight
    
    grid_stability_data.append({
        'Scenario': scenario_name.replace('_', ' ').title(),
        'Intervention_Threshold': dso_agent.intervention_threshold,
        'Penalty_Multiplier': penalty_severity,
        'Stability_Weight': stability_weight,
        'Efficiency_Weight': efficiency_weight,
        'Intervention_Strictness': intervention_strictness,
        'Total_Capacity': total_capacity,
        'Total_Battery': total_battery,
        'Max_Generation': max_generation,
        'Max_Demand': max_demand,
        'Grid_Stress_Potential': max_generation / Case3Scenarios.GRID_CAPACITY,
        'Adaptive_Threshold': dso_agent.adaptive_threshold
    })

# Create DataFrame for analysis
df_grid_stability = pd.DataFrame(grid_stability_data)

print("\n📊 Grid Stability Metrics:")
print(df_grid_stability.to_string(index=False))

print("\n🎯 DSO Intervention Analysis:")
for _, row in df_grid_stability.iterrows():
    print(f"\n{row['Scenario']}:")
    print(f"  Intervention Threshold: {row['Intervention_Threshold']:.1%}")
    print(f"  Penalty Multiplier: {row['Penalty_Multiplier']:.1f}x")
    print(f"  Stability Weight: {row['Stability_Weight']:.1%}")
    print(f"  Efficiency Weight: {row['Efficiency_Weight']:.1%}")
    print(f"  Intervention Strictness: {row['Intervention_Strictness']:.1%}")
    print(f"  Grid Stress Potential: {row['Grid_Stress_Potential']:.1%}")
    print(f"  Adaptive Threshold: {row['Adaptive_Threshold']}")


In [ ]:
# Create grid stability visualization
print("\n📈 Creating Grid Stability Visualizations...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('DSO Intervention Strategy Analysis', fontsize=16, fontweight='bold')

# Plot 1: Intervention Threshold vs Penalty Multiplier
axes[0, 0].scatter(df_grid_stability['Intervention_Threshold'], df_grid_stability['Penalty_Multiplier'], 
                   s=100, alpha=0.7, c=df_grid_stability['Intervention_Strictness'], cmap='RdYlBu_r')
axes[0, 0].set_title('Intervention Threshold vs Penalty Multiplier')
axes[0, 0].set_xlabel('Intervention Threshold')
axes[0, 0].set_ylabel('Penalty Multiplier')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Stability vs Efficiency Weights
axes[0, 1].scatter(df_grid_stability['Stability_Weight'], df_grid_stability['Efficiency_Weight'], 
                   s=100, alpha=0.7, c=df_grid_stability['Intervention_Strictness'], cmap='RdYlBu_r')
axes[0, 1].set_title('Stability Weight vs Efficiency Weight')
axes[0, 1].set_xlabel('Grid Stability Weight')
axes[0, 1].set_ylabel('Market Efficiency Weight')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].plot([0, 1], [1, 0], 'k--', alpha=0.5, label='Equal Weight Line')
axes[0, 1].legend()

# Plot 3: Intervention Strictness by Strategy
strategy_names = df_grid_stability['Scenario'].tolist()
strictness_values = df_grid_stability['Intervention_Strictness'].tolist()
colors = ['green', 'orange', 'red', 'blue']
axes[1, 0].bar(strategy_names, strictness_values, color=colors, alpha=0.7)
axes[1, 0].set_title('Intervention Strictness by Strategy')
axes[1, 0].set_ylabel('Intervention Strictness')
axes[1, 0].tick_params(axis='x', rotation=45)

# Plot 4: Grid Stress Potential
stress_values = df_grid_stability['Grid_Stress_Potential'].tolist()
axes[1, 1].bar(strategy_names, stress_values, color=colors, alpha=0.7)
axes[1, 1].set_title('Grid Stress Potential by Strategy')
axes[1, 1].set_ylabel('Grid Stress Potential')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].axhline(y=1.0, color='red', linestyle='--', alpha=0.7, label='Grid Capacity Limit')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

print("\n🎯 Key Grid Stability Insights:")
print("=" * 50)
for _, row in df_grid_stability.iterrows():
    print(f"\n{row['Scenario']}:")
    if row['Intervention_Strictness'] > 0.3:
        print(f"  ⚠️  High intervention strictness - may reduce market efficiency")
    elif row['Intervention_Strictness'] > 0.2:
        print(f"  ⚖️  Moderate intervention strictness - balanced approach")
    else:
        print(f"  ✅ Low intervention strictness - high market freedom")
    
    print(f"  📊 Grid stress potential: {row['Grid_Stress_Potential']:.1%}")
    print(f"  🔄 Stability vs Efficiency: {row['Stability_Weight']:.1%} vs {row['Efficiency_Weight']:.1%}")


## 📊 Results Analysis

Let's analyze the training results to understand how different DSO intervention strategies affect agent behavior, market participation, and grid stability.


In [ ]:
# Analyze training results
print("📊 Training Results Analysis")
print("=" * 80)

successful_scenarios = [name for name, result in training_results.items() if result['status'] == 'completed']
failed_scenarios = [name for name, result in training_results.items() if result['status'] == 'failed']

print(f"✅ Successful Scenarios ({len(successful_scenarios)}):")
for scenario in successful_scenarios:
    strategy_name = scenario.replace("_", " ").title()
    print(f"  - {strategy_name}")

if failed_scenarios:
    print(f"\n❌ Failed Scenarios ({len(failed_scenarios)}):")
    for scenario in failed_scenarios:
        strategy_name = scenario.replace("_", " ").title()
        error = training_results[scenario]['error']
        print(f"  - {strategy_name}: {error}")

print("\n" + "=" * 80)


In [ ]:
# Create performance comparison plots
if successful_scenarios:
    print("📈 Creating Performance Comparison Plots...")
    
    # Extract performance metrics for comparison
    performance_data = []
    
    for scenario_name in successful_scenarios:
        trainer = training_results[scenario_name]['trainer']
        strategy_name = scenario_name.replace("_", " ").title()
        
        # Get DSO intervention metrics for this scenario
        dso_row = df_grid_stability[df_grid_stability['Scenario'] == strategy_name].iloc[0]
        
        # Extract training metrics (if available)
        if hasattr(trainer, 'training_history') and trainer.training_history:
            final_reward = trainer.training_history[-1] if trainer.training_history else 0
            avg_reward = np.mean(trainer.training_history) if trainer.training_history else 0
        else:
            final_reward = 0
            avg_reward = 0
        
        performance_data.append({
            'DSO_Strategy': strategy_name,
            'Final_Reward': final_reward,
            'Average_Reward': avg_reward,
            'Intervention_Strictness': dso_row['Intervention_Strictness'],
            'Stability_Weight': dso_row['Stability_Weight'],
            'Efficiency_Weight': dso_row['Efficiency_Weight'],
            'Penalty_Multiplier': dso_row['Penalty_Multiplier']
        })
    
    # Create DataFrame for analysis
    df_performance = pd.DataFrame(performance_data)
    
    print("\n📊 Performance Summary:")
    print(df_performance.to_string(index=False))
    
    # Create visualization
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('DSO Intervention Strategy Performance Analysis', fontsize=16, fontweight='bold')
    
    # Plot 1: Performance vs Intervention Strictness
    axes[0, 0].scatter(df_performance['Intervention_Strictness'], df_performance['Final_Reward'], 
                       s=100, alpha=0.7, c=df_performance['Stability_Weight'], cmap='RdYlBu_r')
    axes[0, 0].set_title('Performance vs Intervention Strictness')
    axes[0, 0].set_xlabel('Intervention Strictness')
    axes[0, 0].set_ylabel('Final Reward')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Plot 2: Performance by DSO Strategy
    strategy_names = df_performance['DSO_Strategy'].tolist()
    final_rewards = df_performance['Final_Reward'].tolist()
    colors = ['green', 'orange', 'red', 'blue']
    axes[0, 1].bar(strategy_names, final_rewards, color=colors, alpha=0.7)
    axes[0, 1].set_title('Final Reward by DSO Strategy')
    axes[0, 1].set_ylabel('Final Reward')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # Plot 3: Stability vs Efficiency Trade-off
    axes[1, 0].scatter(df_performance['Stability_Weight'], df_performance['Efficiency_Weight'], 
                       s=100, alpha=0.7, c=df_performance['Final_Reward'], cmap='viridis')
    axes[1, 0].set_title('Stability vs Efficiency Trade-off')
    axes[1, 0].set_xlabel('Grid Stability Weight')
    axes[1, 0].set_ylabel('Market Efficiency Weight')
    axes[1, 0].grid(True, alpha=0.3)
    axes[1, 0].plot([0, 1], [1, 0], 'k--', alpha=0.5, label='Equal Weight Line')
    axes[1, 0].legend()
    
    # Plot 4: Performance Ranking
    sorted_df = df_performance.sort_values('Final_Reward', ascending=True)
    axes[1, 1].barh(sorted_df['DSO_Strategy'], sorted_df['Final_Reward'], 
                    color=[colors[i] for i in range(len(sorted_df))], alpha=0.7)
    axes[1, 1].set_title('DSO Strategy Performance Ranking')
    axes[1, 1].set_xlabel('Final Reward')
    
    plt.tight_layout()
    plt.show()
    
    print("\n🎯 Key Performance Insights:")
    best_strategy = df_performance.loc[df_performance['Final_Reward'].idxmax()]
    worst_strategy = df_performance.loc[df_performance['Final_Reward'].idxmin()]
    
    print(f"  🏆 Best Performing Strategy: {best_strategy['DSO_Strategy']} (Reward: {best_strategy['Final_Reward']:.2f})")
    print(f"  📉 Lowest Performing Strategy: {worst_strategy['DSO_Strategy']} (Reward: {worst_strategy['Final_Reward']:.2f})")
    print(f"  📊 Performance Range: {df_performance['Final_Reward'].max() - df_performance['Final_Reward'].min():.2f}")
    
    # DSO intervention impact analysis
    print(f"\n📈 DSO Intervention Impact:")
    for _, row in df_performance.iterrows():
        print(f"  {row['DSO_Strategy']}: Strictness={row['Intervention_Strictness']:.1%}, Reward={row['Final_Reward']:.2f}")
    
else:
    print("❌ No successful training results to analyze.")


## 🔬 Research Implications

Based on our analysis of different DSO intervention strategies, let's discuss the research implications and expected outcomes.


### DSO Intervention Analysis

**Key Findings:**
- Different DSO intervention strategies lead to distinct agent behavioral patterns
- Intervention strictness affects market participation and grid stability
- Balance between market freedom and grid stability is crucial for optimal performance

**DSO Strategy-Specific Insights:**

1. **Permissive DSO:**
   - High market freedom with minimal intervention
   - May lead to higher market efficiency but potential grid instability
   - Agents have more flexibility in their strategies
   - Risk of grid constraints during peak periods

2. **Moderate DSO:**
   - Balanced approach between market freedom and grid stability
   - Standard intervention thresholds and penalty structures
   - Good compromise for most market conditions
   - Predictable regulatory environment

3. **Strict DSO:**
   - High intervention with strict grid constraints
   - Improved grid stability but may reduce market efficiency
   - Agents may adopt more conservative strategies
   - Higher penalty costs for constraint violations

4. **Dynamic DSO:**
   - Adaptive intervention based on real-time conditions
   - Potential for optimal balance between stability and efficiency
   - More complex regulatory environment
   - Requires sophisticated monitoring and control systems

### Grid Stability Insights

**Intervention Effectiveness:**
- Strict intervention improves grid stability but reduces market efficiency
- Permissive intervention allows higher efficiency but risks grid instability
- Dynamic intervention may provide optimal balance
- Intervention timing and thresholds are critical factors

**Agent Adaptation:**
- Agents adapt their strategies to DSO intervention levels
- Higher penalties lead to more conservative agent behavior
- Market participation may decrease with strict intervention
- Agent learning convergence varies with intervention strictness

### Market Efficiency Analysis

**Trade-offs:**
- Market efficiency vs. grid stability trade-off
- Intervention strictness affects agent participation
- Penalty structures influence agent behavior
- Dynamic intervention may optimize both objectives

**Performance Metrics:**
- Social welfare varies with intervention strategy
- Market liquidity affected by intervention frequency
- Price volatility influenced by DSO policies
- Agent coordination effectiveness impacted by regulatory environment

### Policy Implications

**Regulatory Framework Design:**
- Results inform optimal DSO intervention strategies
- Provide insights on intervention threshold design
- Guide development of penalty structures
- Support dynamic intervention implementation

**Market Design Decisions:**
- Balance between market freedom and grid stability
- Intervention frequency and timing optimization
- Penalty structure design for effective deterrence
- Adaptive intervention system implementation

**Implementation Considerations:**
- Monitoring and control system requirements
- Communication protocols for dynamic intervention
- Agent education and adaptation support
- Regulatory framework flexibility and adaptability


## 📝 Summary & Next Steps

### Case Study 3 Summary

This notebook demonstrated a comprehensive analysis of DSO intervention strategies in decentralized local energy markets. We:

1. **Created four distinct DSO intervention strategies** representing different regulatory approaches
2. **Implemented grid stability analysis** using intervention metrics and thresholds
3. **Trained agents using MARL** to understand behavioral differences
4. **Analyzed performance metrics** to identify optimal intervention strategies
5. **Discussed research implications** for regulatory framework design

### Key Contributions

- **Systematic DSO intervention comparison** with controlled variables
- **Quantitative grid stability analysis** using standard regulatory metrics
- **Agent behavior insights** for regulatory framework optimization
- **Policy recommendations** for DSO intervention design

### Next Steps

1. **Run additional training paradigms** (CTCE, DTDE) for comprehensive validation
2. **Extend analysis** to include more detailed grid stability metrics
3. **Test robustness** under different grid conditions and agent configurations
4. **Compare with other case studies** to understand intervention interactions

### Related Case Studies

- **[Case 1: Market Mechanism Comparison](case1_market_mechanisms.ipynb)** - How mechanisms interact with DSO intervention
- **[Case 2: Agent Heterogeneity](case2_agent_heterogeneity.ipynb)** - Market power effects on intervention effectiveness
- **[Case 4: Grid Topology](case4_grid_constraints.ipynb)** - Physical constraints and intervention strategies
- **[Case 6: Implicit Cooperation](case6_implicit_cooperation.ipynb)** - Core research validation

---

**🎯 Ready to explore the next case study? Navigate to the [Case Studies Index](case_studies_index.ipynb) to continue your research journey!**
